In [103]:
from requests_html import HTMLSession
session = HTMLSession()
from bs4 import BeautifulSoup
import re


In [104]:
fake_keywords = ['fake', 'false', 'misleading', 'misleads', 'unproven', 'no basis', 'inaccurate', 'needs context', 'falsely', 'not']

In [105]:
from collections import defaultdict


vera_dic = defaultdict(list)


def get_quote_text(article_url):
    print('getting quote')
    r2 = session.get(article_url)
    text_body = r2.html.find('article')
    for item in text_body:
        quote = item.find('blockquote')
        if quote:
            quote = quote[0].text
            print(quote)
            fil = re.sub(r'\([^)]*\)', "", quote).replace('"', '')
            try:
                eng = re.search(r'\((.*?)\)',quote).group(1).replace('"', '')
            except:
                eng = None

            if eng == None:
                eng = fil
                fil = None
            
            return fil, eng
            
        else:
            return 'no quotes'



#gives all articles on a verafiles fact check page
article_idx = 0
for i in range(1, 50):
    url = f'https://verafiles.org/section/fact-check/page/{i}'
    r = session.get(url)
    articles = r.html.find('article')
    print(f'page {i}')
    for idx, item in enumerate(articles):
        article_idx = article_idx + 1
        print(f'===article{article_idx}===')
        article = item.find('.article_title.heading-size-2')
        #grabs article title to check if it has been labeled false
        headline = article[0].text.lower() 
        print(headline)
        #if article is labeled false then it will take its url
        if any(word in headline for word in fake_keywords):
            element_str = str(article[0].find('a')[0])
            article_url = re.search("(?P<url>https?://[^\s]+)", element_str).group("url")[:-1]
            quote = get_quote_text(article_url)
            if type(quote) is tuple:
                vera_dic['Article_Headline'].append(headline)
                vera_dic['Statement_Filipino'].append(quote[0])
                vera_dic['Statement_English'].append(quote[1])
                vera_dic['Rating'].append('Fake')
                print(article_url)
            else:
                print(f'===={quote}====')
                print(article_url)
                continue
            

        else:
            element_str = str(article[0].find('a')[0])
            article_url = re.search("(?P<url>https?://[^\s]+)", element_str).group("url")[:-1]
            quote = get_quote_text(article_url)
            if type(quote) is tuple:
                vera_dic['Article_Headline'].append(headline)
                vera_dic['Statement_Filipino'].append(quote[0])
                vera_dic['Statement_English'].append(quote[1])
                vera_dic['Rating'].append('Real')
                print(article_url)  
            else:
                print(f'===={quote}====')
                print(article_url)  
                continue  
                   



page 1
===article1===
vera files fact check: marcos, tulfo not closing down naia
getting quote
“KAKAPASOK LANG SAWAKAS (sic) NAIA SARADO NA! SEN TULFO SUMABOG sa GALIT NILUSOB ang NAIA mga BUWAYA IYAK (Just in, NAIA’s closed! Sen. Tulfo exploded in anger, charged thru NAIA. Crocodiles cry).”
https://verafiles.org/articles/vera-files-fact-check-marcos-tulfo-not-closing-down-naia'
===article2===
vera files fact check: claim about a ph-us deal to build a nuclear facility misleads
getting quote
“Breaking news! Pilipinas, magkaka-nuclear na! Biden, nakaplano na ang uumpisahang deal kasama si President Ferdinand Marcos Jr. tungkol sa itatayong nuclear facility sa Pilipinas (The Philippines will now have nuclear [energy]! [U.S. President Joe] Biden has planned a deal with President Ferdinand Marcos Jr. about a nuclear facility to be built in the Philippines)!”
https://verafiles.org/articles/vera-files-fact-check-claim-about-a-ph-us-deal-to-build-a-nuclear-facility-misleads'
===article3===
ver

In [106]:
vera_dic

defaultdict(list,
            {'Article_Headline': ['vera files fact check: marcos, tulfo not closing down naia',
              'vera files fact check: claim about a ph-us deal to build a nuclear facility misleads',
              'vera files fact check: false video links hontiveros to 2014 philhealth bonus controversy anew',
              'vera files fact check: us, ph soldiers did not expel chinese ships from south china sea',
              'vera files fact check: marcos did not shut down meralco',
              'vera files fact check: doctor on tiktok claim that oral glutathione helps reverse fatty liver is unproven',
              'vera files fact check: video falsely claims cayetano has been removed as senator',
              'vera files fact check: marcos sr.’s false bar exam grade reappears after release of 2022 bar results',
              'vera files fact check: ‘super meal mix nuts’ ads to fight diabetes, cancer are a scam',
              'vera files fact check: video does not 

In [107]:
import pandas as pd
df = pd.DataFrame(vera_dic)
df = df.fillna("")
df

,Article_Headline,Statement_Filipino,Statement_English,Rating
0,"vera files fact check: marcos, tulfo not closi...",“KAKAPASOK LANG SAWAKAS NAIA SARADO NA! SEN T...,sic,Fake
1,vera files fact check: claim about a ph-us dea...,"“Breaking news! Pilipinas, magkaka-nuclear na!...",The Philippines will now have nuclear [energy]...,Fake
2,vera files fact check: false video links honti...,“KAKAPASOK LANG PRES MARCOS MARCOLETA HAWAK an...,JUST IN PRES MARCOS MARCOLETA HAVE THE DOCUMEN...,Fake
3,"vera files fact check: us, ph soldiers did not...","“US AT PILIPINAS, NAGPADALA NG 17,680 NA SUNDA...","U.S. and the Philippines sent 17,680 soldiers ...",Fake
4,vera files fact check: marcos did not shut dow...,“KAKAPASOK LANG OMG! mga OLlGARKO NAPRANlNG na...,JUST IN OMG! OLIGARCHS BECAME PARANOID MERALCO...,Fake
...,...,...,...,...
155,vera files fact check: escudero inaccurately c...,“May mga nagsusulong ng pederalismo sa ating b...,There are people pushing for federalism in the...,Fake
156,vera files fact check: imee marcos falsely cla...,“Walang nagugutom na bata noon kay Nutribun an...,na,Fake
157,vera files fact check: imee marcos’ claim that...,"“But at the same time, it is very clear, China...",it should be,Fake
158,"vera files fact check: for the nth time, badoy...",“We attribute these votes to the treachery of ...,archive,Real


In [108]:
df.to_csv('VeraFilesDataset.csv', encoding='utf-8-sig', index=False)

In [109]:
df = pd.read_csv('VeraFilesDataset.csv',delimiter=',')

In [110]:
df = df.fillna("")
df = df.applymap(lambda x: x.replace('"', ''))
df

,Article_Headline,Statement_Filipino,Statement_English,Rating
0,"vera files fact check: marcos, tulfo not closi...",“KAKAPASOK LANG SAWAKAS NAIA SARADO NA! SEN T...,sic,Fake
1,vera files fact check: claim about a ph-us dea...,"“Breaking news! Pilipinas, magkaka-nuclear na!...",The Philippines will now have nuclear [energy]...,Fake
2,vera files fact check: false video links honti...,“KAKAPASOK LANG PRES MARCOS MARCOLETA HAWAK an...,JUST IN PRES MARCOS MARCOLETA HAVE THE DOCUMEN...,Fake
3,"vera files fact check: us, ph soldiers did not...","“US AT PILIPINAS, NAGPADALA NG 17,680 NA SUNDA...","U.S. and the Philippines sent 17,680 soldiers ...",Fake
4,vera files fact check: marcos did not shut dow...,“KAKAPASOK LANG OMG! mga OLlGARKO NAPRANlNG na...,JUST IN OMG! OLIGARCHS BECAME PARANOID MERALCO...,Fake
...,...,...,...,...
155,vera files fact check: escudero inaccurately c...,“May mga nagsusulong ng pederalismo sa ating b...,There are people pushing for federalism in the...,Fake
156,vera files fact check: imee marcos falsely cla...,“Walang nagugutom na bata noon kay Nutribun an...,na,Fake
157,vera files fact check: imee marcos’ claim that...,"“But at the same time, it is very clear, China...",it should be,Fake
158,"vera files fact check: for the nth time, badoy...",“We attribute these votes to the treachery of ...,archive,Real
